## 프로젝트 주제 : 맛집 탐방
https://www.data.go.kr/data/15066516/fileData.do

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/모함ㅋ/용산구_맛집.csv", encoding='cp949')  
df.head(20)

In [ ]:
import folium
center = [37.541, 126.986]
m = folium.Map(location=center, zoom_start=11, tiles="Stamenterrain")

m.add_child(folium.ClickForMarker()) # popup이 없으면 위도,경도 표시

In [ ]:
# step1, step2 기능 구현
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')
# 위도, 경도 반환하는 함수
def geocoding(address):
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

    
hyunho_address_name = input() #이태원동
seoyoon_address_name = input() #우사단로4길

# step 1
hyunho_address = geocoding(hyunho_address_name) # [37.5270966, 127.0015525]
seoyoon_address = geocoding(seoyoon_address_name) # [어쩌구,저쩌구]

# step 2 : 중심 지점
average_address = [(hyunho_address[0]+seoyoon_address[0])/2 , (hyunho_address[1]+seoyoon_address[1])/2]

m = folium.Map(location = center, zoom_start=15)
    
#지도에 데이터 찍어서 보여주기
folium.Marker(hyunho_address, tooltip=hyunho_address_name).add_to(m)
folium.Marker(seoyoon_address, tooltip=seoyoon_address_name).add_to(m)
folium.Marker(average_address, tooltip = '중심').add_to(m)

# 거리 계산
print(haversine(hyunho_address, seoyoon_address, unit = 'm'))
m

ㅇ# step3 : 중심지에서 가까운(100m 이내) 맛집 위치 찾기

In [ ]:
# 중심지까지의 거리 구하기

df2 = pd.DataFrame.copy(df)
df2['중심지까지의 거리'] = None
for i in range(len(df)):
  lat = df.iloc[i,7]
  lon = df.iloc[i,8]
  dis = haversine([lat, lon], average_address, unit='m')
  df2.iloc[i,10] = dis

df2.head()

In [ ]:
df2[df2['중심지까지의 거리'] < 100]

In [ ]:
map = folium.Map(location = average_address, zoom_start=16)
df_100 = df2[df2['중심지까지의 거리'] < 100]
for i in df_100.index:
    sub_lat =  df_100.loc[i,'위도']
    sub_long = df_100.loc[i,'경도']
    
    title = df_100.loc[i,'업소명']
    menu = df_100.loc[i, '주요요리']
    
    #지도에 데이터 찍어서 보여주기
    folium.Marker([sub_lat,sub_long],tooltip = title, popup = menu).add_to(map)
    # folium.Marker([sub_lat,sub_long], popup = menu).add_to(map)

folium.Marker(hyunho_address,tooltip = hyunho_address_name, icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
folium.Marker(seoyoon_address,tooltip = seoyoon_address_name, icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
folium.Marker(average_address,tooltip = '중심',icon=folium.Icon(icon = "cloud",color = "green")).add_to(map)
# folium.Marker(hyunho_address,icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
# folium.Marker(seoyoon_address,icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
# folium.Marker(average_address,icon=folium.Icon(icon = "cloud",color = "green")).add_to(map)

# Circle = 미터단위 /  CircleMarker = 픽셀단위(확대, 축소를 해도 원의 크기가 비율에 맞춰 일정하게유지)
folium.Circle(location=average_address, radius=100, color="blue", fill=True).add_to(map)

map

# Step4. 먹기 싫은 음식 걸러내기
- 먹기 싫은 음식의 키워드가 들어간 메뉴를 파는 가게는 보여주지 않는다

In [ ]:
df3 = pd.DataFrame.copy(df2)
# for menu in ['소고기','비프','규','한우']:
#   df_cow = df3[df3['주요요리'].str.contains(menu)]
#   print(df_cow.주요요리)

# for menu in ['소고기','비프','규','한우']:
#   df_cow = df3[df3['주요요리'].str.contains(menu)]
#   df_not_cow = df3.drop(df_cow)
#   # print(df_cow.주요요리)
# print(df_not_cow)
def to_nan(x):
  for menu in ['소고기','비프','규','한우']:
    if menu in x:
      return np.NaN
  
  return x
    

df_not_cow = pd.DataFrame.copy(df3)
df_not_cow['주요요리'] = df_not_cow['주요요리'].apply(lambda x: to_nan(x))
df_not_cow_2 = df_not_cow[df_not_cow['주요요리'].notna()]  # notna : True, False 값으로 나옴

In [ ]:
df_not_cow_2[df_not_cow_2['중심지까지의 거리'] < 100]

In [ ]:
map = folium.Map(location = average_address, zoom_start=16)
df_not_cow_100 = df_not_cow_2[df_not_cow_2['중심지까지의 거리'] < 100]
for i in df_not_cow_100.index:
    sub_lat =  df_not_cow_100.loc[i,'위도']
    sub_long = df_not_cow_100.loc[i,'경도']
    
    title = df_not_cow_100.loc[i,'업소명']
    menu = df_not_cow_100.loc[i, '주요요리']
    
    #지도에 데이터 찍어서 보여주기
    folium.Marker([sub_lat,sub_long],tooltip = title, popup = menu).add_to(map)
    # folium.Marker([sub_lat,sub_long], popup = menu).add_to(map)

folium.Marker(hyunho_address,tooltip = hyunho_address_name, icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
folium.Marker(seoyoon_address,tooltip = seoyoon_address_name, icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
folium.Marker(average_address,tooltip = '중심',icon=folium.Icon(icon = "cloud",color = "green")).add_to(map)
# folium.Marker(hyunho_address,icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
# folium.Marker(seoyoon_address,icon=folium.Icon(icon = "info-sign",color = "red")).add_to(map)
# folium.Marker(average_address,icon=folium.Icon(icon = "cloud",color = "green")).add_to(map)

# Circle = 미터단위 /  CircleMarker = 픽셀단위(확대, 축소를 해도 원의 크기가 비율에 맞춰 일정하게유지)
folium.Circle(location=average_address, radius=100, color="blue", fill=True).add_to(map)

map

# Step5 : 크롤링 - Selenium
### 문제점 
- 네이버 플레이스에 없는 음식점이 많다
- 가게마다 메뉴 표시가 다르다
- 동명의 가게가 많다
### 해결방안
- 망고플레이트(https://www.mangoplate.com/)에서 크롤링 진행
- 각종 맛집에 대한 정보와 음식 사진을 가져올 수 있음

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)


url = "https://www.mangoplate.com/restaurants/z7EYi8LIox"
driver.get(url)
element = driver.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/ul/li[1]/a/div[2]/ul/li[1]/button/img")

img_url = element.get_attribute("data-src")
img_url = img_url.split('?')[0]
print(img_url)
driver.close()

import os
import time
from PIL import Image

# time check
start = time.time()

# curl 요청
# curl "이미지 주소" > "저장 될 이미지 파일 이름" 
os.system("curl " + img_url + " > test1.jpg")

# 이미지 다운로드 시간 체크
print(time.time() - start)


# 저장 된 이미지 확인
curl_img = Image.open("/content/test1.jpg")
curl_img

# Step 6. 딥러닝으로 음식사진 데이터 Classification
### 출처 : kaggle https://www.kaggle.com/rkuo2000/food11-classification/data

train_dir = "/content/gdrive/MyDrive/모함ㅋ/training"
val_dir   = "/content/gdrive/MyDrive/모함ㅋ/validation"
test_dir  = "/content/gdrive/MyDrive/모함ㅋ/evaluation"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_size = (224,224) # 임의로 줘도 된다. 단 딥러닝 들어갈때 shape를 맞춰주기만 하면 된다
# 보편적으로 224x224로 이미지 학습
batch_size = 16 # batch = epoch를 한번에 넣으면 학습시간이 오래걸려서 나눠서 넣는다.
# 이미지 16개씩 학습할 것이다. -> 정확도는 좀 떨어지지만 학습속도 향상

# Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255, # RGB(0,0,0) 픽셀값 정규화
#    rotation_range=40, # 데이터 증식(증강) 과정 : 주석 풀면서 더 향상된 학습 하기 위해 
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
#    zoom_range=0.2,
   horizontal_flip=True,
   vertical_flip=True
)

# path의 데이터를 ImageDataGenerator로 불러와주는 우리만의 함수를 만들자!
# def get_dataset(path, datagen, shuffle):
#     data_set = datagen.flow_from_directory(path,
#                                            target_size=target_size,
#                                            batch_size=batch_size, # 한 번에 이미지 두 장씩
#                                            class_mode='categorical', # cats, dogs 두 개면 binary를 쓰기도 함.
#                                            color_mode='rgb',
#                                            shuffle=shuffle, # True or False
#                                            seed=42)   # data shuffle시에 쓰는 난수값(얼마나 shuffle할 것이냐?)                  

#     return data_set

# train_generator = get_dataset(train_dir, train_datagen, True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical') # 각 사진에 대한 train_y label 을 매칭시켜줄 필요는 없을
    # 것으로 보임. ImageDataGenerator가 알아서 분류를 해줌([['Bread'], ['Bread'].,..]) 이런 식


In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
target_size = (224,224) # 임의로 줘도 된다. 단 딥러닝 들어갈때 shape를 맞춰주기만 하면 된다
# 보편적으로 224x224로 이미지 학습
batch_size = 16
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='categorical')

print(test_generator) #class_indices 한번 보기
print(list(test_generator.class_indices))

In [ ]:
labels = list(test_generator.class_indices.keys())
# print(labels)
print(test_generator.class_indices)
print(test_generator.classes)
# 궁금증 해결!